In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split


c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_source = pd.read_csv('data_source.csv')
#data_source = data_source[data_source['label']=='153 - Input Data Manipulation']
data_source, _ = train_test_split(data_source, test_size=0.9, random_state=42)
data_target  = pd.read_csv('data_target.csv')
data_target, _ = train_test_split(data_target, test_size=0.9, random_state=42)

data_target = data_target.sample(frac=1, random_state=42)
#data_target = data_target[data_target['label']=='153 - Input Data Manipulation']
X_source = data_source['text'].str.replace('/',' ')
y_source = data_source['label']
X_target  = data_target ['text'].str.replace('/','')
y_target  = data_target ['label']
rlist =['194 - Fake the Source of Data', '66 - SQL Injection',
       '34 - HTTP Response Splitting', '126 - Path Traversal',
       '000 - Normal', '272 - Protocol Manipulation',
       '310 - Scanning for Vulnerable Software', '242 - Code Injection',
       '153 - Input Data Manipulation']
mapping = {l: i+1 for i, l in enumerate(rlist)}
y_source = [mapping[s] for s in y_source] 
y_target  = [mapping[r] for r in y_target ]
y_source = np.array(y_source)
y_target  = np.array(y_target )


In [3]:
len(X_source)

28148

In [4]:
import multiprocessing

In [5]:
# from transformers import AutoTokenizer, AutoModel


# X_source = list(X_source)
# X_target = list(X_target)
# input_texts_source = X_source
# input_text_target = X_target
# model_name = 'jackaduma/SecBERT'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# def extract_features(text):
#     tokens = tokenizer.tokenize(text)
#     token_ids = tokenizer.convert_tokens_to_ids(tokens)
#     input_tensor = torch.tensor([token_ids])

#     with torch.no_grad():
#         outputs = model(input_tensor)
    
#     last_hidden_state = outputs.last_hidden_state
#     first_token_features = last_hidden_state[0, 0, :]

#     return first_token_features

In [6]:
from transformers import AutoTokenizer, AutoModel
model_name = 'jackaduma/SecBERT'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

X_source = list(X_source)
X_target = list(X_target)
input_texts_source = X_source
input_text_target = X_target

X_train = []
X_test = []

for i , text in enumerate(input_texts_source):

    tokens = tokenizer.tokenize(text)

    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_tensor = torch.tensor([token_ids])
    with torch.no_grad():
        outputs = model(input_tensor)

    last_hidden_state = outputs.last_hidden_state

    first_token_features = last_hidden_state[0, 0, :]

    X_train.append(first_token_features)
    if i%1000==0 :
        print(i/len(X_source))

for j , text in enumerate(input_text_target):

    tokens = tokenizer.tokenize(text)

    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_tensor = torch.tensor([token_ids])
    with torch.no_grad():
        outputs = model(input_tensor)

    last_hidden_state = outputs.last_hidden_state

    first_token_features = last_hidden_state[0, 0, :]

    X_test.append(first_token_features)
    if j%1000==0 :
        print(j/len(X_target))


Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.0
0.03552650277106722
0.07105300554213444
0.10657950831320165
0.14210601108426887
0.17763251385533607
0.2131590166264033
0.24868551939747052
0.28421202216853775
0.3197385249396049
0.35526502771067214
0.39079153048173937
0.4263180332528066
0.4618445360238738
0.49737103879494104
0.5328975415660082
0.5684240443370755
0.6039505471081427
0.6394770498792098
0.6750035526502771
0.7105300554213443
0.7460565581924116
0.7815830609634787
0.817109563734546
0.8526360665056132
0.8881625692766804
0.9236890720477476
0.9592155748188148
0.9947420775898821
0.0
0.12297097884899164
0.24594195769798327
0.36891293654697493
0.49188391539596654
0.6148548942449582
0.7378258730939499
0.8607968519429414
0.9837678307919331


In [7]:
X_test = torch.stack(X_test)
X_train = torch.stack(X_train)

In [8]:
y_target = np.where(np.logical_or(y_target == 8, y_target == 9), 0, 1)

In [9]:
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# import json


# class Vectorizer:
#     def __init__(self, method='BOW', ngram_range=(1, 1), max_features=300, emb_fname='', word_index_fname=''):
#         self.method = method
#         if self.method == 'BOW':
#             self.vectorizer = CountVectorizer(analyzer='word', input='content', ngram_range=ngram_range, max_features=max_features)
#         elif self.method == 'TFIDF':
#             self.vectorizer = TfidfVectorizer(analyzer='word', input='content', max_features=max_features)
#         elif self.method == 'Word2Vec':
#             self.max_features = max_features
#             self.emb_fname = emb_fname
#             self.word_index_fname = word_index_fname
#         else:
#             raise ValueError('Feature extraction method does not exist.')

#     def feature_extraction(self, X_train, X_test):
#         train_data = self.vectorizer.fit_transform(X_train).toarray()
#         test_data = self.vectorizer.transform(X_test).toarray()
#         return train_data, test_data

#     def get_word_index(self):
#         word2id = json.load(open(self.word_index_fname, 'r'))
#         return word2id

#     def get_embedding_matrix(self):
#         np.random.seed(0)
#         word2id = self.get_word_index()
#         embedding_matrix = np.random.uniform(-0.25, 0.25, [len(word2id) + 1, self.max_features])
#         with open(self.emb_fname, 'r', encoding='utf-8') as f:
#             for line in f:
#                 content = line.split(' ')
#                 if content[0] in word2id:
#                     embedding_matrix[word2id[content[0]]] = np.array(list(map(float, content[1:])))
#         return embedding_matrix

In [10]:
# vectorizer = Vectorizer('TFIDF')
# X_train, X_test = vectorizer.feature_extraction(X_source, X_target)

In [11]:
# from gensim.models import Word2Vec
# sentences = [sentence.split() for sentence in X_source]
# w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

In [12]:
# def vectorize(sentence):
#     words = sentence.split()
#     words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
#     if len(words_vecs) == 0:
#         return np.zeros(100)
#     words_vecs = np.array(words_vecs)
#     return words_vecs.mean(axis=0)

# X_train = np.array([vectorize(sentence) for sentence in X_source])
# X_test = np.array([vectorize(sentence) for sentence in X_target])

In [13]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [14]:
input_layer = X_train.shape[1]

In [15]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_layer, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), input_layer)
        output = self.model(x)
        return output

In [16]:
discriminator = Discriminator().to(device=device)

In [17]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_layer, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_layer),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), input_layer)
        return output

generator = Generator().to(device=device)

In [18]:
lr = 0.00001
num_epochs = 1000
batch_size= 32
loss_function = nn.BCELoss()
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [19]:
X_source = X_train.to(device=device, dtype=torch.float32)
X_target = X_test.to(device=device, dtype=torch.float32)


In [20]:
# X_source = torch.from_numpy(X_train).to(device=device, dtype=torch.float32)
# X_target = torch.from_numpy(X_test).to(device=device, dtype=torch.float32)

In [21]:
for epoch in range(num_epochs):
    idx = np.random.randint(0, X_source.shape[0], batch_size)
    real_samples  = X_source[idx]
    real_samples_labels = torch.ones((batch_size,1)).to(device=device)
    latent_space_samples = torch.rand((batch_size,input_layer)).to( device=device)
    
    generated_samples = generator(latent_space_samples)
    generated_samples_labels = torch.zeros((batch_size, 1)).to(
            device=device
        )
    all_samples = torch.cat((real_samples, generated_samples))
    all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
    discriminator.zero_grad()
    output_discriminator = discriminator(all_samples)
    loss_discriminator = loss_function(
            output_discriminator, all_samples_labels
        )
    loss_discriminator.backward()
    optimizer_discriminator.step()

        # Data for training the generator
    latent_space_samples = torch.rand((batch_size,input_layer)).to(device=device)

        # Training the generator
    generator.zero_grad()
    generated_samples = generator(latent_space_samples)
    output_discriminator_generated = discriminator(generated_samples)
    loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
    loss_generator.backward()
    optimizer_generator.step()
    if(epoch%100==0):
        print(f"Epoch: {epoch/100} Loss D.: {loss_discriminator}")
        print(f"Epoch: {epoch/100} Loss G.: {loss_generator}")
 

Epoch: 0.0 Loss D.: 0.7013780474662781
Epoch: 0.0 Loss G.: 0.7065547704696655
Epoch: 1.0 Loss D.: 0.5207404494285583
Epoch: 1.0 Loss G.: 0.6323729157447815
Epoch: 2.0 Loss D.: 0.43727222084999084
Epoch: 2.0 Loss G.: 0.7495318055152893
Epoch: 3.0 Loss D.: 0.4119413495063782
Epoch: 3.0 Loss G.: 0.746780276298523
Epoch: 4.0 Loss D.: 0.35297921299934387
Epoch: 4.0 Loss G.: 0.8493854403495789
Epoch: 5.0 Loss D.: 0.3598809838294983
Epoch: 5.0 Loss G.: 0.7763939499855042
Epoch: 6.0 Loss D.: 0.2969912886619568
Epoch: 6.0 Loss G.: 0.875113844871521
Epoch: 7.0 Loss D.: 0.2880514860153198
Epoch: 7.0 Loss G.: 0.9223963022232056
Epoch: 8.0 Loss D.: 0.3007437288761139
Epoch: 8.0 Loss G.: 0.9320600628852844
Epoch: 9.0 Loss D.: 0.2509166896343231
Epoch: 9.0 Loss G.: 0.987364649772644


In [22]:
predicted_labels = discriminator(X_target)
new_tensor = torch.where(predicted_labels >= 0.5, torch.tensor(1), torch.tensor(0))

In [23]:
new_label = new_tensor.numpy()

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
accuracy = accuracy_score(y_target, new_label)
accuracy

0.8573536645351697